<div align='center'>

# OVERVIEW
</div>

This notebook aims to clarify whether or not any of the suggested Payments & Transfer services are acceptable for use for our Customer Support Center system, as defined in the [`customer_support_rep_persona_01.md` persona document](../../customer_rep_persona_store/customer_support_rep_persona_01.md).

The suggested services are listed below:

1. Ethereum Sepolia Testnet
2. MetaMask
3. Etherscan (Sepolia)

The criteria for qualifying a suggestion as a tool is defined below:

1. Python-based.
2. Open-source (free, avoid freemium as much as possible) customer-facing and producer-facing access.
3. Rate limit restrictions allow for appropriate edge-case testing.
4. Supports core architectural choice for LangChain ecosystem (e.g. supports asynchronous backends, etc).
5.  I can currently access the tool as a human (i.e. confirmation that it is operable as a human, and has not been deprecated/restricted discreetly).
6. I can perform its expected basic operations.

## Pre-Notebook Initialization

1. Setup MetaMask (Google Chrome browser extension).
2. Elicit SepoliaETH from a Sepolia faucet.
3. Verify transaction on [Sepolia Etherscan](sepolia.etherscan.io).

<u>Notes</u>:
- Ethereum addresses are the same across all EVM networks.

In [5]:
# Required Modules

import os
import requests
from dotenv import load_dotenv
load_dotenv()
import pandas as pd

## Etherscan

<ol>
<li> Python-based: Available
<li> Open-source: Free

<li> Maximum Rate Limit Restriction:
<ul>
<li> 5 calls per second.
<li> 100K calls per day.
<li> 3 API keys per account.
</ul>

<li> Supports Core Architectural Choice
<li> Service is confirmed to still be active & accessible
</ol>

### Expected Basic Operations

1. [Get Native Balance for an Address](https://docs.etherscan.io/api-reference/endpoint/balance).
2. [Get Normal Transactions By Address](https://docs.etherscan.io/api-reference/endpoint/txlist).
3. [Get ERC20 Token Transfers by Address](https://docs.etherscan.io/api-reference/endpoint/tokentx).
4. [Get Address Funded By](https://docs.etherscan.io/api-reference/endpoint/fundedby).
5. [Get Event Logs by Address](https://docs.etherscan.io/api-reference/endpoint/getlogs).
6. [Get ERC20-Token Account Balance for TokenContractAddress](https://docs.etherscan.io/api-reference/endpoint/tokenbalance).

**<u>Notes:</u>**
1. Etherscan offers a JSON-RPC API for interacting with its system.

### Setup

Etherscan uses an Remote Procedure Call (RPC) API - it is not RESTful.

In [ ]:
# Class Definition for API calls

class Simple_Etherscan_RPC_API_Client():

    def __init__(
                self, 
                 random_eth_address: str = "0xC257274276a4E539741Ca11b590B9447B26A8051" # Randomly selected (via Google Search) for service confirmation purposes.
                 ):
        
        self.random_eth_address = random_eth_address
        self.SUCCESS_MSG = "[INFO]: Your GET request was been performed successfully!"
        self.ERROR_MSG = "[WARNING]:"
        self.data = None

    def perform_get_request(self, params: dict):
        """Performs GET request"""

        try:
            if "apikey" not in params.keys():
                params["apikey"] = os.getenv("ETHERSCAN_API_KEY")

            response = requests.get(
                url="https://api.etherscan.io/v2/api",
                params=params
            )

            self.data = response.json()
            
            print(self.SUCCESS_MSG)

        except Exception as e:
            print(self.ERROR_MSG, e)
            return 0

        return (f"Value for field 'result' is: {response.json()['result']}")

### 1. Get Native Balance for an Address

Retrieves the native token balance held by a specific address.

In [50]:
# Client 1 Init

etherscan_client1 = Simple_Etherscan_RPC_API_Client()

# Making API call

params1 = {
    "chainid":1,
    "address": etherscan_client1.random_eth_address,
    "module":"account",
    "action": "balance",
    "tag": "latest"
                }

etherscan_client1.perform_get_request(params1)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: 84479637358432397"

In [51]:
# Converting WEI to ETH

wei = int(etherscan_client1.data['result'])
eth = wei / 1018
print(f"Total ETH Amount: {eth}")

Total ETH Amount: 0.08447963735843239


### 2. Get Normal Transactions By Address

Retrieves the transaction history of a specified address, with optional pagination.

In [52]:
# Client 2 Init
etherscan_client2 = Simple_Etherscan_RPC_API_Client()


params2 = {
    "chainid":"1",
    "module": "account",
    "action": "txlist",
    "address": etherscan_client2.random_eth_address,
    "startblock": 0,
    "endblock": 9999999999,
    "page": 1,
    "offset": 1,
    "sort": "desc"
}

etherscan_client2.perform_get_request(params2)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: [{'blockNumber': '23883863', 'blockHash': '0x0790935e7f7fd1a25dd8beda47bcdeb0f341f60bdcdf8661fa6c4bd78c3e09fd', 'timeStamp': '1764171563', 'hash': '0xc33d26c762c6527c300526abbfa092849d87595cc5609bc2f6ef853333b9d48c', 'nonce': '7', 'transactionIndex': '149', 'from': '0xd1bd16dba8daca7923eb064dd2be29afc115e87e', 'to': '0xc257274276a4e539741ca11b590b9447b26a8051', 'value': '265815149418', 'gas': '21000', 'gasPrice': '83014813', 'input': '0x', 'methodId': '0x', 'functionName': '', 'contractAddress': '', 'cumulativeGasUsed': '31047601', 'txreceipt_status': '1', 'gasUsed': '21000', 'confirmations': '16089', 'isError': '0'}]"

### 3. Get ERC20 Token Transfers by Address

Retrieves the list of ERC-20 token transfers made by a specified address, with optional filtering by token contract.

In [53]:
# Client 3 Init

etherscan_client3 = Simple_Etherscan_RPC_API_Client()

params3 = {
    "chainid": "1",
    "module": "account",
    "action": "tokentx",
    "contractaddress": "0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2",
    "address": "0x4e83362442b8d1bec281594cea3050c8eb01311c",
    "startblock": 0,
    "endblock": 9999999999,
    "page": 1,
    "offset": 1,
    "sort": "desc"
}

etherscan_client3.perform_get_request(params3)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: [{'blockNumber': '15572639', 'timeStamp': '1663652063', 'hash': '0x62afa8a25e569f9659e35856284e8a674fadb3029f522d0556ac6ec01adb38ad', 'nonce': '1579', 'blockHash': '0x530b533d29e73ca8376eb1298199f52fa0f34d43f2d381febaf02312453e19ec', 'from': '0x4e83362442b8d1bec281594cea3050c8eb01311c', 'contractAddress': '0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2', 'to': '0xe8c6c9227491c0a8156a0106a0204d881bb7e531', 'value': '163529846812879587', 'tokenName': 'Maker', 'tokenSymbol': 'MKR', 'tokenDecimal': '18', 'transactionIndex': '62', 'gas': '254396', 'gasPrice': '6650574109', 'gasUsed': '181367', 'cumulativeGasUsed': '4085175', 'input': 'deprecated', 'methodId': '0x5ae401dc', 'functionName': 'multicall(uint256 deadline, bytes[] data)', 'confirmations': '8327314'}]"

### 4. Get Address Funded By

Retrieves the address and transaction that first funded a specific address, useful for tracing fund origins.

In [54]:
# Client 4 Init
etherscan_client4 = Simple_Etherscan_RPC_API_Client()

params4 = {
    "chainid":"1",
    "module":"account",
    "action": "fundedby",
    "address": "0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97"
}

etherscan_client3.perform_get_request(params4)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: {'block': 16976894, 'timeStamp': '1680628103', 'fundingAddress': '0x21a31ee1afc51d94c2efccaa2092ad1028285549', 'fundingTxn': '0xb5b9e8281c99d88258931da08a67650a40684d9d81d3c01d759b1442a0734e85', 'value': '300000000000000000'}"

### 5. Get Event Logs by Address

Retrieves event logs from a specific address, with optional block range filtering.

In [55]:
# Client 5 Init
etherscan_client5 = Simple_Etherscan_RPC_API_Client()

params5 = {
    "chainid": "1",
    "module": "logs",
    "action": "getLogs",
    "address": "0xbd3531da5cf5857e7cfaa92426877b022e612cf8",
    "fromBlock": 12878196,
    "toBlock": 12878196,
    "page": 1,
    "offset": 1000
}

etherscan_client5.perform_get_request(params5)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: [{'address': '0xbd3531da5cf5857e7cfaa92426877b022e612cf8', 'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', '0x0000000000000000000000000000000000000000000000000000000000000000', '0x000000000000000000000000c45a4b3b698f21f88687548e7f5a80df8b99d93d', '0x00000000000000000000000000000000000000000000000000000000000000b5'], 'data': '0x', 'blockNumber': '0xc48174', 'blockHash': '0x837e109ab8b1b40ec7d1032bff82397325d85e719b97d900fa0d9aa9745b2c27', 'timeStamp': '0x60f9ce56', 'gasPrice': '0x2e90edd000', 'gasUsed': '0x247205', 'logIndex': '0x', 'transactionHash': '0x4ffd22d986913d33927a392fe4319bcd2b62f3afe1c15a2c59f77fc2cc4c20a9', 'transactionIndex': '0x'}, {'address': '0xbd3531da5cf5857e7cfaa92426877b022e612cf8', 'topics': ['0x645f26e653c951cec836533f8fe0616d301c20a17153debc17d7c3dbe4f32b28', '0x00000000000000000000000000000000000000000000000000000000000000b5'], 'data': '0x', 'blockNumber': '0xc48174', 'blockHash': '0x837e109ab8b1b40e

### 6. Get ERC20-Token Account Balance for TokenContractAddress

Retrieves the current ERC-20 token balance for a specified address.

In [57]:
# Client 6 Init

etherscan_client6 = Simple_Etherscan_RPC_API_Client()

params6 = {
    "chainid": "1",
    "module": "account",
    "action": "tokenbalance",
    "contractaddress": "0x57d90b64a1a57749b0f932f1a3395792e12e7055",
    "address": "0xe04f27eb70e025b78871a2ad7eabe85e61212761",
    "tag": "latest"
}

etherscan_client6.perform_get_request(params=params6)

[INFO]: Your GET request was been performed successfully!


"Value for field 'result' is: 135499"

## MetaMask

### <u>Pre-requisite Information</u>

MetaMask has partnered with Infura to offer a comprehensive set of services to assist with dApp and Snap development, including JSON-RPC APIs for easy access to key networks, and REST APIs aimed at automating and optimizing essential development tasks. 


<ol>
<li> Python-based: <b>Available</b>
<li> Open-source: <b>Free</b>

<li> Rate Limit Restrictions (dependent on if the request is I) <a href="https://docs.metamask.io/services/concepts/websockets#pricing">a <b>WebSocket streaming</b> request</a>, II) <a href="https://docs.metamask.io/services/how-to/make-batch-requests">a <b>batch</b> request</a>, or III) <a href="https://docs.metamask.io/services/concepts/archive-data">an <b>archive</b> request</a>):

<ol type="i">
<li> Credit costs are broken down further in the following <a href="https://docs.metamask.io/services/get-started/pricing/credit-cost/">link</a>.

<ol type="a">
<li> <b>Free Tier</b>: 3M Daily Credits, 500 credits per second rate limit.
<li> <b>Developer Tier</b>: 15M Daily Credits, 4K credits per second rate limit.
<li> <b>Team Tier</b>: 75M Daily Credits, 40K credits per second rate limit.
<li> <b>Custom</b>: (Contact MetaMask Team)
</ol>

</ol>

<li> Supports core architectural choice.
<li> Service is confirmed to <b>still be active & accessible</b>.
</ol>

### <u>Expected Basic Operations</u>

1. [Send Transactions](https://docs.metamask.io/services/tutorials/ethereum/send-a-transaction/send-a-transaction-py).
2. [Get Normal Transactions By Address](https://docs.etherscan.io/api-reference/endpoint/txlist).
3. [Get ERC20 Token Transfers by Address](https://docs.etherscan.io/api-reference/endpoint/tokentx).
4. [Get Address Funded By](https://docs.etherscan.io/api-reference/endpoint/fundedby).
5. [Get Event Logs by Address](https://docs.etherscan.io/api-reference/endpoint/getlogs).
6. [Get ERC20-Token Account Balance for TokenContractAddress](https://docs.etherscan.io/api-reference/endpoint/tokenbalance).

**<u>Notes:</u>**
<ol>

<!-- 1 -->
<li> MetaMask partners with Infura to deliver a comprehensive set of services to facilitate dApp and Snap development, including i) <b>JSON-RPC APIs</b> for easy access to key networks, and ii) <b>REST APIs</b> aimed at automating and optimizing essential development tasks e.g. the <a href='https://docs.metamask.io/services/reference/gas-api'>Gas API</a>.

<!-- 2 -->
<li> The following process is used to determine the credit cost for each request:

<ul>
<li> Initial check for a specific credit value.
<li> Fallback to the Ethereum method's credit value.
<li> Fallback to the default credit value.
</ul>

<!-- 3 -->
<li> Requests that return HTTP status code errors have their own credit costs:

<ul>
<li> <code>429</code> errors: These are not charged and can occur if you've exceeded your allowed throughput limit (requests per second).
<li> <code>402</code> errors: These are not charged and can occur if you've exceeded your allowed daily credit usage.
<li> <code>4xx</code> errors: These are errors caused by human input, and consume 5 credits.
<li> <code>5xx</code> errors: These are server errors, and do not consume any credit charges.
</ul>

<!-- 4 -->
<li> Daily credit quota limits are reset every 00:00 UTC.

</ol>

### <u>Notes</u>

- After a conversation with MetaMask's FinAI, I have understood MetaMask's role in this to indeed be **customer-facing, but more of a manual way of managing one's account** rather than a programmatic way - I will maintain this method for such and rather focus on familiarizing myself with it in a manual manner.